In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import cv2
import seaborn
import cv2
from PIL import Image
from tqdm import tqdm

In [2]:
images = np.load('/mnt/storage1/Hover-Net/Fold 1/images/fold1/images.npy',allow_pickle=True)
masks = np.load('/mnt/storage1/Hover-Net/Fold 1/masks/fold1/masks.npy',allow_pickle=True)

print(images.shape)
print(masks.shape)

(2656, 256, 256, 3)
(2656, 256, 256, 6)


In [3]:
out_dir = "./"

# A helper function to map 2d numpy array
def flat_for(a, f):
    a = a.reshape(-1)
    for i, v in enumerate(a):
        a[i] = f(v)


# A helper function to unique PanNuke instances indexes to [0..N] range where 0 is background
def map_inst(inst):
    seg_indexes = np.unique(inst)
    new_indexes = np.array(range(0, len(seg_indexes)))
    dict = {}
    for seg_index, new_index in zip(seg_indexes, new_indexes):
        dict[seg_index] = new_index

    flat_for(inst, lambda x: dict[x])


# A helper function to transform PanNuke format to HoverNet data format
def transform(images, masks, path, out_dir, start, finish):

    fold_path = out_dir+path
    try:
        os.mkdir(fold_path)
    except FileExistsError:
        pass
    
    start = int(images.shape[0]*start)
    finish = int(images.shape[0]*finish)
    
    for i in tqdm(range(start, finish)):
        np_file = np.zeros((256,256,5), dtype='int16')

        # add rgb channels to array
        img_int = np.array(images[i],np.int16)
        for j in range(3):
            np_file[:,:,j] = img_int[:,:,j]

        # convert inst and type format for mask
        msk = masks[i]

        inst = np.zeros((256,256))
        for j in range(5):
            #copy value from new array if value is not equal 0
            inst = np.where(msk[:,:,j] != 0, msk[:,:,j], inst)
        map_inst(inst)

        types = np.zeros((256,256))
        for j in range(5):
            # write type index if mask is not equal 0 and value is still 0
            types = np.where((msk[:,:,j] != 0) & (types == 0), j+1, types)

        # add padded inst and types to array
        np_file[:,:,3] = inst
        np_file[:,:,4] = types

        np.save(fold_path + '/' + '%d.npy' % (i), np_file)

In [4]:
transform(images, masks, 'train', out_dir=out_dir, start=0, finish=0.8)

100%|██████████| 2124/2124 [00:42<00:00, 49.47it/s]


In [5]:
transform(images, masks, 'val', out_dir=out_dir, start=0.8, finish=1)

100%|██████████| 532/532 [00:10<00:00, 49.91it/s]
